In [1]:
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import datetime

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [2]:
# Diretório para os pontos coletados
DATA_DIR = os.path.join('Dados_extraidos') 

# Sinais
sinais = np.array(['Ola','Tudo bem','Obrigado','Entendo','Calma'])

# Quantidade de vídeos por sinal
num_videos = 600

# Quantidade de frames por vídeo
num_frames = 30

#Quantidade de coordenadas por frame
num_coordenadas = 1662

classe_dic = {classe:num for num, classe in enumerate(sinais)}
videos, classes = [], []

In [ ]:
import keras_tuner as kt

def build_model(hp):

  model = Sequential()

  model.add(LSTM(units=hp.Int("units", min_value=64, max_value=1024, step=64), activation='relu', input_shape=(30,1662)))

  for i in range(hp.Int("num_layers", 1, 3)):
    model.add(Dense(units=hp.Int("units", min_value=64, max_value=1024, step=64), activation='relu'))

  model.add(Dense(5, activation='softmax'))

  learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
  
  model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

  return model

In [ ]:
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="categorical_accuracy",
    max_trials=10,
    executions_per_trial=1,
    overwrite=True,
    directory="Searchs",
    project_name="arquitecture",
)

In [3]:
# Configura a rede neural

model = Sequential()
model.add(LSTM(512, activation='relu', dropout=0.2, input_shape=(num_frames,num_coordenadas)))
model.add(Dense(512, activation='relu'))
model.add(Dense(sinais.shape[0], activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [4]:
# Monta os vídeos e classes

for sinal in sinais:

    lista_videos = os.listdir(os.path.join(DATA_DIR, sinal))

    for video_num in np.array(lista_videos).astype(int):

        video = []

        for frame_num in range(num_frames):
            frame_npy = os.path.join(DATA_DIR, sinal, str(video_num), "{}.npy".format(frame_num))
            frame = np.load(frame_npy)
            video.append(frame)

        videos.append(video)
        classes.append(classe_dic[sinal])

In [1]:
# Cria variáveis de treino, validação e teste

X = np.array(videos)
Y = to_categorical(classes).astype(int)
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, 
                                                        Y, 
                                                        test_size=0.2,
                                                        stratify=Y)
X_teste, X_val, Y_teste, Y_val = train_test_split(X_teste,
                                                Y_teste,
                                                test_size=0.5,
                                                stratify=Y_teste)
print(X_treino.shape)
print(X_val.shape)
print(X_teste.shape)
print(Y_treino.shape)
print(Y_val.shape)
print(Y_teste.shape)

NameError: name 'np' is not defined

In [6]:
print(np.unique(np.argmax(Y_treino, axis=1)))
print(np.unique(np.argmax(Y_teste, axis=1)))
print(np.unique(np.argmax(Y_val, axis=1)))

[0 1 2 3 4]
[0 1 2 3 4]
[0 1 2 3 4]


In [7]:
# Cria pasta de logs e seta os callbacks

log_dir = os.path.join('Logs', '6')
tensorboard = TensorBoard(log_dir=log_dir)
reduceLR = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.1, patience=10)
early_stopping = EarlyStopping(monitor='val_categorical_accuracy', patience=50, restore_best_weights=True)

In [8]:
model.fit(X_treino, 
        Y_treino, 
        epochs=120, 
        callbacks=[tensorboard, early_stopping, reduceLR], 
        validation_data=(X_val, Y_val))

Epoch 1/120
75/75 [==============================] - 32s 402ms/step - loss: 244.0064 - categorical_accuracy: 0.2179 - val_loss: 71.4594 - val_categorical_accuracy: 0.1167
Epoch 2/120
75/75 [==============================] - 31s 414ms/step - loss: 19.1018 - categorical_accuracy: 0.2417 - val_loss: 1.9983 - val_categorical_accuracy: 0.2567
Epoch 3/120
75/75 [==============================] - 31s 411ms/step - loss: 1.4264 - categorical_accuracy: 0.4062 - val_loss: 1.1433 - val_categorical_accuracy: 0.5400
Epoch 4/120
75/75 [==============================] - 31s 413ms/step - loss: 1.0248 - categorical_accuracy: 0.5487 - val_loss: 0.7976 - val_categorical_accuracy: 0.6067
Epoch 5/120
75/75 [==============================] - 30s 401ms/step - loss: 0.8506 - categorical_accuracy: 0.6246 - val_loss: 0.6540 - val_categorical_accuracy: 0.7333
Epoch 6/120
75/75 [==============================] - 30s 407ms/step - loss: 0.8617 - categorical_accuracy: 0.6388 - val_loss: 0.5918 - val_categorical_accur

In [9]:
# Resumo do modelo
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 512)               4454400   
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 2565      
Total params: 4,719,621
Trainable params: 4,719,621
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Testa o modelo
model.evaluate(X_teste, Y_teste)

10/10 [==============================] - 2s 166ms/step - loss: 0.0624 - categorical_accuracy: 0.9833


[0.06244867295026779, 0.9833333492279053]

In [ ]:
# Salva os pesos
model.save('pesosbons.h5')